In [ ]:
기술지표

In [ ]:
입력 데이터가 매우 중요함.. 
사용하는 데이터에 따라 주가를 사용해 기술지표를 만듬
퀀트 - 기술지표, 기업재무제표 - 가치투자
올라간 주식이 계속오른다 - 모멘텀
올라간 주식은 반드시 내려온다(또는 평균으로 회귀함) - 평균회귀지표 
기술적 지표중 자주사용하는것 
- 이동평균선, - 상대 강도 지수, - 스토캐스틱 오실레이터 등 
주식 시장을 바라보는 트레이더의 시각에 따라 사용되는 지표가 다름 
가치투자전략 : 당기순이익, 영업이익, 영업이익률, 매출액, 부채비율,PER,PBR,PSR,
    PCR,ROE.ROA등 재무제표 데이터를 기초로함.. 
주가 데이터를 사용해 만든 기술 지표를 활용하는 방법처럼 지표를 계산하는 작업보다
기준이 되는 데이터에서 순위를 매겨 분위별로 자르는 작업이 훨씬 중요함 

In [ ]:
평균회귀지표 

In [ ]:
평균으로서의 회귀 : 평균에 가까워지려는 경향성 
회귀지표를 볼린저밴드로 구현해봄
볼린저밴드 : 중심선(이동평균선), 상단선과 하단선(표준편차밴드)
추세와 변동성을 분석해줌.. 
보통 중심선을 이루는 이동평균선을 계산할때 20일을 사용
상하위밴드는 20일 이동평균선 +- 2*20일 이동 표준편차를 사용함 
ex) 상단밴드 = 중간밴드 +2*20일 이동 표준편차
    중간밴드 = 20일 이동 평균선
    하단밴드 = 중간밴드 -2*20일 이동 표준편차 

In [ ]:
볼린저밴드 구현
import pandas as pd
df = pd.read_csv('./data/ETFs_main.csv')
df.head()
가장먼저 해야할 작업은 데이터를 불러와 탐색하는것임.
주식차트를 볼때 확인하는것 
날짜, 시가, 고가, 저가, 종가, 수정종가, 거래량 데이터로 구성됨 
describe라는 함수를 이용해 데이터셋의 분포,중심,경향 분산 및 형태를 
요약하는 통계 정보를 확인할수있음

모든 변수를 사용할 필요가 없으므로 필요한 변수만 선택하기 위해 
슬라이싱, 인덱싱, 서브셋을 이용해 날짜, 수정종가를 선택 및 추출하고
copy함수를 통해 복사함
price_df = df.loc[:,['Date','Adj Close']].copy()
price_df.head()

날짜, 수정 종가를 추출한 데이터 프레임을 price_df 변수에 저장함 
연,월,일형식의 'Date'컬럼을 price_df의 인덱스로 설정할것임 
price_df.set_index(['Date'],inplace=True)
price_df.head()

볼린저밴드를 구현하려면 중간밴드를 먼저 만들어야함으로 20일이동평균선을
.rolling()함수로 구함
price_df['center']=price_df['Adj Close'].rolling(20).mean()
price_df.iloc[18:25]
상단밴드 만들기
price_df['ub'] = price_df['center'] +2 *price_df['Adj Close'],rolling(20).std()
price_df.iloc[18:25] #mean()이라는 함수대신 표준편차를 구하는 std()를 사용
price_df['lb'] = price_df['center'] -2 *price_df['Adj Close'].rolling(20).std()
price_df.iloc[18:25]

함수를 지정한다 
입력 매개변수로 이동 평균선 기간과 표준 편차값을 전달받는다
n = 20
sigma = 2
def bollinger_band(price_df,n,sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(20).mean()
    bb['ub'] = price_df['Adj Close'] + sigma * price_df['Adj Close'].rolling(n).std()
    bb['lb'] = price_df['Adj Close'] - sigma * price_df['Adj Close'].rolling(n).std()    
    
    return bb
bollinger = bollinger_band(price_df,n,sigma)


In [ ]:
기간을 나눠 볼린저 밴드를 활용한 성과를 확인한다
base_date = '2009-01-02'
sample = bollinger.loc[base_date:]